# Agglomerative hierarchical clustering

In this technique, each data point is a cluster in the begginning. Then, join the two closest data points,
based on some distance, into one cluster. Run this process until one big cluster is formed. Then, 
divide the resulting hierarchical clustering into the desired number of clusters.

## Methodology

### Number of groups

Agglomerative clustering allows to define the number of groups at the end of the execution. This work will
consider $N \in \{2,3,\ldots,13\}$ groups. Since the algorithm is deterministic, only one executing per $N$
value will be performed.

### Davies-Bouldin (DB) index

Every clustering will then be evaluated in the view of the Davies-Bouldin (DB) index:

$$
R_{ij} = \frac{s_i + s_j}{d_{ij}}
$$

$$
D_i = \max_{j\neq i}R_{ij}
$$

$$
DB = \frac 1 k \sum_i D_i
$$

## Implementation

### Function to load the datasets

In [ ]:
import pandas as pd

def load_dataset(paths):
    '''
    Load the datasets in paths.
    '''
    # load train and test datasets together
    dataset = pd.concat([pd.read_csv(f) for f in paths])
    # keep targets for future comparative purposes
    targets = dataset['cover_type']
    # remove target column
    dataset.drop('cover_type', inplace=True, axis=1)
    # check shape
    print("[INFO] Dataset shape: ", dataset.shape)
    # return
    return dataset

### Function to run the clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score

def perform_agg_clustering(dataset, cluster_sizes):
    '''
    Perform aggregation clustering.
    '''
    results = []
    models = {}
    for size in cluster_sizes:
        print("[INFO] Running for size", size)
        # train model
        cluster = AgglomerativeClustering(n_clusters=size, affinity='euclidean', linkage='ward')
        cluster.fit(dataset)
        # compute db
        db = davies_bouldin_score(dataset, cluster.labels_)
        # store result
        results.append([size, db])
        # store models
        models[size] = cluster
        print("[INFO] Done for size", size, "db =",db)
    return (models, pd.DataFrame(results, columns=['cluster_size','db_index']))

### Function to save models and results

In [ ]:
from sklearn.externals import joblib

def save_models_results(results, models, save_model_path, save_results_path):
    '''
    Save clustering models and results.
    '''
    for size in models:
        joblib.dump(models[size], save_model_path + '_' + str(size))
    results.to_csv(save_results_path, index=False)

### Function to perform the entire pipeline

In [ ]:
def perform_and_save_agg(paths, cluster_sizes, save_model_path, save_results_path):
    '''
    Run clustering and save models and results.
    '''
    dataset = load_dataset(paths)
    models, df_results = perform_agg_clustering(dataset, cluster_sizes)
    save_models_results(df_results, models, save_model_path, save_results_path)
    return df_results

### Execute for the given datasets

In [ ]:
sizes = list(range(2,14))
# original dataset
original_res = perform_and_save_agg(['../datasets/covertype_norm_train.csv', 
                      '../datasets/covertype_norm_test.csv'],
                    sizes, 
                    '../models/agg_clustering_original.save',
                    '../results/agg_clustering_original.csv')
# lda dataset
lda_res = perform_and_save_agg(['../datasets/covertype_lda_train.csv', 
                       '../datasets/covertype_lda_test.csv'],
                    sizes, 
                    '../models/agg_clustering_lda.save',
                    '../results/agg_clustering_lda.csv')

## Results

Results are expressed in terms of the DB index:

### Original

In [ ]:
original_res

In [ ]:
original_res.plot.line(x='cluster_size', y='db_index')

### LDA

In [ ]:
lda_res

In [ ]:
lda_res.plot.line(x='cluster_size', y='db_index')